In [1]:
import sys
import pandas as pd
from lxml import etree
import re
from unidecode import unidecode

print(sys.executable)

/opt/anaconda3/envs/pouille/bin/python


## Preprocessing

1. extration des articles à lier et des features utiles dans df
2. chargement du référentiel des communes (COG 2011) dans le df main_insee_commune

In [2]:
# Extraction des features de chaque article à lier dans un df

xml_file = '../../data/PO_t9/PO_t9.xml'
xml_tree = etree.parse(xml_file)

data = []

i = 0 # compter les articles

for article in xml_tree.xpath('//article'):
    i+=1
    # on ne retient pas les lieux hors France
    if article.find('./localisationpa') is not None:
        continue
    vedettes = article.xpath('./vedette/i')
    # on ne retient que la première vedette
    vedette = vedettes[0].text if vedettes else ''

    # on ne retient que le premier dpt de localisation
    localisationde = article.find('./localisationde')
    localisationde = localisationde.text if localisationde is not None else 'none'

    # on ne retient que le premier canton de localisation
    localisationca = article.find('./localisationca')
    if localisationca is not None:
        localisationca = re.search(
            r'<localisationca>(.*?)<\/localisationca>',
            etree.tostring(localisationca, encoding=str)
        ).group(1)
        '''
        cases:
        - 'c<sup>on</sup> de '
        - 'c<sup>on</sup> du '
        - 'c<sup>on</sup> d’'
        - 'c<sup>on</sup> de la '
        - 'c<sup>on</sup> de l’'
        - 'c<sup>on</sup> des '
        '''
        localisationca = re.sub(r"c<sup>on</sup> (de |d’|de la |du |de l’|des )", '', localisationca)
    else:
        localisationca = 'none'

    #  on ne retient que la première commune de localisation
    localisationco = article.find('./localisationco')
    if localisationco is not None:
        localisationco = re.search(
            r'<localisationco>(.*?)<\/localisationco>',
            etree.tostring(localisationco, encoding=str)
        ).group(1)
        '''
        cases:
        - 'c<sup>on</sup> de '
        - 'c<sup>on</sup> du '
        - 'c<sup>on</sup> d’'
        - 'c<sup>on</sup> de la '
        - 'c<sup>on</sup> de l’'
        - 'c<sup>on</sup> des '
        - 'c<sup>nes</sup> de '
        - 'c<sup>nes</sup> d’'
            '''
        localisationco = re.sub(r"c<sup>ne(s)?</sup> (de |d’|de la |du |de l’|des )", '', localisationco)

    else:
        localisationco = 'none'

    # Extraire le contenu textuel de la balise "article"
    article_text = re.sub(r'\W', ' ', article.xpath('string(.)'))

    # on insère tout dans un dictionnaire
    data.append({
        'old-id': article.get('old-id'),
        'vedette': vedette,
        'localisationde': localisationde,
        'localisationca': localisationca,
        'localisationco': localisationco,
        'reference': article_text  # Ajouter la colonne "reference"
    })

df = pd.DataFrame(data)

'''
article : nombre d’articles dans la source => `//article`
article_to_link : nombre d’articles à lier (en France) => `//article[not(localisationpa)]`
article_commune : nombre d’articles de type commune => `//article[not(localisationpa) and not(localisationco)]`
article_loc_com : nombre d’articles localisés dans une commune => `//article[not(localisationpa) and localisationco]`
'''
report = dict(
    article = i,
    article_to_link = len(df.index),
    article_commune = (df.localisationco == 'none').sum(),
    article_loc_com = (df.localisationco != 'none').sum()
)

# check
[print(k,':',v) for k, v in report.items()]
df.iloc[1500:1510, :]

article : 5321
article_to_link : 5321
article_commune : 3487
article_loc_com : 1834


,old-id,vedette,localisationde,localisationca,localisationco,reference
1500,PO9-01522,Égletons,Corrèze,Tulle,none,Égletons Corrèze con de Tulle ...
1501,PO9-01523,Église-aux-Bois,Corrèze,Treignac,none,Église aux Bois L Corrèze con ...
1502,PO9-01524,Égliseneuve-d’Entraigues,Puy-de-Dôme,Besse,none,Égliseneuve d Entraigues Puy de Dôme ...
1503,PO9-01525,Égliseneuve-des-Liards,Puy-de-Dôme,Sauxillanges,none,Égliseneuve des Liards Puy de Dôme ...
1504,PO9-01526,Égliseneuve-près-Billom,Puy-de-Dôme,Billom,none,Égliseneuve près Billom Puy de Dôme ...
1505,PO9-01527,Églisolles,Puy-de-Dôme,Viverols,none,Églisolles Puy de Dôme con de Viv...
1506,PO9-01528,Éguzon,Indre,La Châtre,none,Éguzon Indre con de La Châtre ...
1507,PO9-01529,Elbes,Aveyron,Villefranche-de-Rouergue,Martiel,Elbes Aveyron con de Villefranche...
1508,PO9-01530,Endevaysse,Corrèze,Sornac,Saint-Germain-Lavolp,Endevaysse Corrèze con de Sornac ...
1509,PO9-01531,Ennezat,Puy-de-Dôme,Riom,none,Ennezat Puy de Dôme con de Riom ...


In [3]:
# Copiez les colonnes d'origine
df['vedette_orig'] = df['vedette']
df['localisationco_orig'] = df['localisationco']

In [4]:
#premier préprocessing pour les caractères spéciaux

In [5]:
file_path = '../../utils/pouille/resources/tokens_dicotopo.txt'

with open(file_path, 'r') as file:
    tokens = file.read()

In [6]:
def replace_special_characters(text):
    text = unidecode(text)
    text = re.sub(r'[-\'\[\]\(\)]', ' ', text)
    return text


df['vedette'] = df['vedette'].apply(replace_special_characters)
df['localisationco'] = df['localisationco'].apply(replace_special_characters)

In [7]:
df

,old-id,vedette,localisationde,localisationca,localisationco,reference,vedette_orig,localisationco_orig
0,PO9-00001,Abbas,Aveyron,Rodez,Druelle,Abbas Aveyron con de Rodez cn...,Abbas,Druelle
1,PO9-00002,Abbaye Nouvelle,Lot,Salviac,Leobard,Abbaye Nouvelle L Lot con de...,Abbaye[-Nouvelle],Léobard
2,PO9-00003,Abeille,Haute-Vienne,Châlus,Chalus,Abeille L Haute Vienne con de ...,Abeille,Châlus
3,PO9-00004,Abjat,Dordogne,Nontron,none,Abjat Dordogne con de Nontron ...,Abjat,none
4,PO9-00005,Abloux,Indre,Saint-Benoît-du-Sault,Saint Gilles,Abloux Indre con de Saint Benoît ...,Abloux,Saint-Gilles
...,...,...,...,...,...,...,...,...
5316,PO9-05397,Begoux,Lot,Cahors,Cahors,Bégoux Lot con de Cahors cne ...,Bégoux,Cahors
5317,PO9-05398,Chauzas,Corrèze,Vigeois,Perpezac le Noir,Chauzas Corrèze con de Vigeois ...,Chauzas,Perpezac-le-Noir
5318,PO9-05399,Neuglise,Allier,Neuilly-le-Réal,Bessay sur Allier,Neuglise Allier con de Neuilly le...,Neuglise,Bessay-sur-Allier
5319,PO9-05400,Roc Le,Corrèze,Larche,Saint Pantaleon de Larche,Roc Le Corrèze con de Larche ...,Roc (Le),Saint-Pantaléon-de-Larche


In [8]:
# Ajout du dpt code au df
dpt_df = pd.read_csv('../../utils/pouille/resources/departements-region.csv')
dpt_dict = dict(zip(dpt_df['dep_name'], dpt_df['num_dep']))

# on ajoute notre colonne avec les numéros des départements qui matchent
df['dpt_code'] = df['localisationde'].apply(lambda x: dpt_dict.get(x, 'none'))

#on renomme et réordonne pour plus de clarté.
df = df.rename(columns={'old-id' : 'article_id', 'localisationca' : 'canton_code', 'localisationco_present' : 'nom_commune'})
df = df[['article_id', 'vedette', 'localisationde', 'dpt_code', 'canton_code', 'localisationco', 'reference', 'vedette_orig', 'localisationco_orig']]

df.iloc[1500:1510, :]

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig
1500,PO9-01522,Egletons,Corrèze,19,Tulle,none,Égletons Corrèze con de Tulle ...,Égletons,none
1501,PO9-01523,Eglise aux Bois,Corrèze,19,Treignac,none,Église aux Bois L Corrèze con ...,Église-aux-Bois,none
1502,PO9-01524,Egliseneuve d Entraigues,Puy-de-Dôme,63,Besse,none,Égliseneuve d Entraigues Puy de Dôme ...,Égliseneuve-d’Entraigues,none
1503,PO9-01525,Egliseneuve des Liards,Puy-de-Dôme,63,Sauxillanges,none,Égliseneuve des Liards Puy de Dôme ...,Égliseneuve-des-Liards,none
1504,PO9-01526,Egliseneuve pres Billom,Puy-de-Dôme,63,Billom,none,Égliseneuve près Billom Puy de Dôme ...,Égliseneuve-près-Billom,none
1505,PO9-01527,Eglisolles,Puy-de-Dôme,63,Viverols,none,Églisolles Puy de Dôme con de Viv...,Églisolles,none
1506,PO9-01528,Eguzon,Indre,36,La Châtre,none,Éguzon Indre con de La Châtre ...,Éguzon,none
1507,PO9-01529,Elbes,Aveyron,12,Villefranche-de-Rouergue,Martiel,Elbes Aveyron con de Villefranche...,Elbes,Martiel
1508,PO9-01530,Endevaysse,Corrèze,19,Sornac,Saint Germain Lavolp,Endevaysse Corrèze con de Sornac ...,Endevaysse,Saint-Germain-Lavolp
1509,PO9-01531,Ennezat,Puy-de-Dôme,63,Riom,none,Ennezat Puy de Dôme con de Riom ...,Ennezat,none


In [9]:
# Chargement du référentiel des communes
fields = ['insee_code', 'DEP_id', 'NCCENR']
main_insee_commune = pd.read_csv("../../utils/pouille/resources/main_insee_commune.tsv",
                                 delimiter='\t',
                                 usecols=fields,
                                 dtype={'insee_code': 'string'})
main_insee_commune['DEP_id'] = main_insee_commune['DEP_id'].apply(lambda x: x[4:])
main_insee_commune.head(5)

,insee_code,DEP_id,NCCENR
0,01001,01,Abergement-Clémenciat
1,01002,01,Abergement-de-Varey
2,01004,01,Ambérieu-en-Bugey
3,01005,01,Ambérieux-en-Dombes
4,01006,01,Ambléon


In [10]:
main_insee_commune['NCCENR_orig'] = main_insee_commune['NCCENR']

In [11]:
main_insee_commune['NCCENR'] = main_insee_commune['NCCENR'].apply(replace_special_characters)

In [12]:
main_insee_commune.head(5)

,insee_code,DEP_id,NCCENR,NCCENR_orig
0,01001,01,Abergement Clemenciat,Abergement-Clémenciat
1,01002,01,Abergement de Varey,Abergement-de-Varey
2,01004,01,Amberieu en Bugey,Ambérieu-en-Bugey
3,01005,01,Amberieux en Dombes,Ambérieux-en-Dombes
4,01006,01,Ambleon,Ambléon


In [13]:
# Liste les départements présents pour mieux filtrer quand pas de localisationde
dpt_list = df['dpt_code'].unique().tolist()
dpt_list.remove("none")
dpt_list.sort()
print(dpt_list)

['03', '06', '07', '11', '12', '13', '15', '16', '18', '19', '23', '24', '30', '31', '32', '33', '34', '36', '37', '38', '41', '42', '43', '45', '46', '47', '48', '49', '58', '63', '71', '81', '82', '86', '87', '89']


## Liage des articles de type commune

In [14]:
# dataframe à charger
linked_places_df = pd.DataFrame()

In [15]:
# Extraction des articles de type commune (ceux qui n’ont pas de commune de localisation)

communes_df = df[df['localisationco'] == 'none']
communes_df.iloc[500:1000, :]
#communes_df.shape

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig
801,PO9-00815,Carlux,Dordogne,24,Sarlat,none,Carlux Dordogne con de Sarlat ...,Carlux,none
803,PO9-00817,Carmaux,Tarn,81,Albi,none,Carmaux Tarn con d Albi arrt ...,Carmaux,none
810,PO9-00824,Cassagnas,Lozère,48,Barre-des-Cévennes,none,Cassagnas Lozère con de Barre des...,Cassagnas,none
811,PO9-00825,Cassagnes,Lot,46,Cazals,none,Cassagnes Lot con de Cazals ...,Cassagnes,none
812,PO9-00826,Cassagnes Begonhes,Aveyron,12,Rodez,none,Cassagnes Begonhès Aveyron con de...,Cassagnes-Begonhès,none
...,...,...,...,...,...,...,...,...,...
1503,PO9-01525,Egliseneuve des Liards,Puy-de-Dôme,63,Sauxillanges,none,Égliseneuve des Liards Puy de Dôme ...,Égliseneuve-des-Liards,none
1504,PO9-01526,Egliseneuve pres Billom,Puy-de-Dôme,63,Billom,none,Égliseneuve près Billom Puy de Dôme ...,Égliseneuve-près-Billom,none
1505,PO9-01527,Eglisolles,Puy-de-Dôme,63,Viverols,none,Églisolles Puy de Dôme con de Viv...,Églisolles,none
1506,PO9-01528,Eguzon,Indre,36,La Châtre,none,Éguzon Indre con de La Châtre ...,Éguzon,none


### Exact match method

#### With dpt

In [16]:
import pandas as pd

# Merge communes_df and main_insee_commune on vedette and dpt_code
communes_exact_dpt_df = pd.merge(communes_df,
                                 main_insee_commune,
                                 how='inner',
                                 left_on=['vedette', 'dpt_code'],
                                 right_on=['NCCENR', 'DEP_id'])

# Create a new column 'insee' by combining NCCENR and insee_code
communes_exact_dpt_df['insee'] = communes_exact_dpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
communes_exact_dpt_df = communes_exact_dpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
communes_exact_dpt_df = communes_exact_dpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'dpt_exact'
communes_exact_dpt_df['method'] = 'dpt_exact'

# Check for duplicate entries
duplicates = communes_exact_dpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    communes_exact_dpt_df = communes_exact_dpt_df[~duplicates]


# Update the linked_places_df dataframe
linked_places_df = pd.concat([linked_places_df, communes_exact_dpt_df]).drop_duplicates()

In [17]:
# Tests
#communes_exact_dpt_df.head(5)
communes_exact_dpt_df

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,NCCENR_orig,insee_code,method
0,PO9-00006,Abrest,Allier,03,Vichy,none,Abrest Allier con de Vichy ...,Abrest,none,Abrest,Abrest (03001),dpt_exact
1,PO9-00007,Abzac,Charente,16,Confolens-sud,none,Abzac Charente con de Confolens s...,Abzac,none,Abzac,Abzac (16001),dpt_exact
2,PO9-00008,Acheres,Cher,18,Henrichemont,none,Achères Cher con d Henrichemont ...,Achères,none,Achères,Acheres (18001),dpt_exact
3,PO9-00009,Affieux,Corrèze,19,Treignac,none,Affieux Corrèze con de Treignac ...,Affieux,none,Affieux,Affieux (19001),dpt_exact
4,PO9-00010,Agen,Lot-et-Garonne,47,none,none,Agen Lot et Garonne Agennense ...,Agen,none,Agen,Agen (47001),dpt_exact
...,...,...,...,...,...,...,...,...,...,...,...,...
3221,PO9-05391,Yronde et Buron,Puy-de-Dôme,63,Vic-le-Comte,none,Yronde et Buron Puy de Dôme con d...,Yronde-et-Buron,none,Yronde-et-Buron,Yronde et Buron (63472),dpt_exact
3222,PO9-05392,Yssac la Tourette,Puy-de-Dôme,63,Combronde,none,Yssac la Tourette Puy de Dôme con...,Yssac-la-Tourette,none,Yssac-la-Tourette,Yssac la Tourette (63473),dpt_exact
3223,PO9-05393,Yssandon,Corrèze,19,Agen,none,Yssandon Corrèze con d Agen ...,Yssandon,none,Yssandon,Yssandon (19289),dpt_exact
3224,PO9-05394,Yssingeaux,Haute-Loire,43,Puy,none,Yssingeaux Haute Loire con du Puy...,Yssingeaux,none,Yssingeaux,Yssingeaux (43268),dpt_exact


In [18]:
# des doublons de liage exact match dans un même dpt ?
#communes_exact_dpt_df.article_id.duplicated().sum()
communes_exact_dpt_df.loc[communes_exact_dpt_df.duplicated(keep=False, subset=['article_id']), :]

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,NCCENR_orig,insee_code,method


#### Without dpt

In [19]:
import pandas as pd

# Merge communes_df and main_insee_commune on vedette and NCCENR
communes_exact_nodpt_df = pd.merge(communes_df.loc[communes_df['dpt_code'] == 'none'],
                                   main_insee_commune[main_insee_commune.DEP_id.isin(dpt_list)],
                                   how='inner',
                                   left_on='vedette',
                                   right_on='NCCENR')

# Create a new column 'insee' by combining NCCENR and insee_code
communes_exact_nodpt_df['insee'] = communes_exact_nodpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
communes_exact_nodpt_df = communes_exact_nodpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
communes_exact_nodpt_df = communes_exact_nodpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'nodpt_exact'
communes_exact_nodpt_df['method'] = 'nodpt_exact'

duplicates = communes_exact_nodpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    communes_exact_nodpt_df = communes_exact_nodpt_df[~duplicates]

# Update the linked_places_df dataframe
linked_places_df = pd.concat([linked_places_df, communes_exact_nodpt_df]).drop_duplicates()

In [20]:
# Test
communes_exact_nodpt_df
#linked_places_df.shape
communes_exact_nodpt_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,NCCENR_orig,insee_code,method
nunique,3,3,1,1,1,1,3,3,1,3,3,1
count,3,3,3,3,3,3,3,3,3,3,3,3
size,3,3,3,3,3,3,3,3,3,3,3,3


In [21]:
communes_exact_nodpt_df.loc[communes_exact_nodpt_df.duplicated(keep=False,subset=['article_id']), :]

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,NCCENR_orig,insee_code,method


## Liage des communes de localisation

### Exact match method

#### With dpt

In [22]:
# Extraction des articles appartenant à une commune (ceux ont une commune de localisation)

pas_communes_df = df[df['localisationco'] != 'none']
pas_communes_df

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig
0,PO9-00001,Abbas,Aveyron,12,Rodez,Druelle,Abbas Aveyron con de Rodez cn...,Abbas,Druelle
1,PO9-00002,Abbaye Nouvelle,Lot,46,Salviac,Leobard,Abbaye Nouvelle L Lot con de...,Abbaye[-Nouvelle],Léobard
2,PO9-00003,Abeille,Haute-Vienne,87,Châlus,Chalus,Abeille L Haute Vienne con de ...,Abeille,Châlus
4,PO9-00005,Abloux,Indre,36,Saint-Benoît-du-Sault,Saint Gilles,Abloux Indre con de Saint Benoît ...,Abloux,Saint-Gilles
11,PO9-00012,Aglan,Lot,46,Puy-l’Évêque,Soturac,Aglan Lot con de Puy l Évêque ...,Aglan,Soturac
...,...,...,...,...,...,...,...,...,...
5315,PO9-05396,Yvernault,Indre,36,Argenton,Mosnay,Yvernault Indre con d Argenton ...,Yvernault,Mosnay
5316,PO9-05397,Begoux,Lot,46,Cahors,Cahors,Bégoux Lot con de Cahors cne ...,Bégoux,Cahors
5317,PO9-05398,Chauzas,Corrèze,19,Vigeois,Perpezac le Noir,Chauzas Corrèze con de Vigeois ...,Chauzas,Perpezac-le-Noir
5318,PO9-05399,Neuglise,Allier,03,Neuilly-le-Réal,Bessay sur Allier,Neuglise Allier con de Neuilly le...,Neuglise,Bessay-sur-Allier


In [23]:
# Merge pas_communes_df and main_insee_commune on localisationco and dpt_code
pas_communes_exact_dpt_df = pd.merge(pas_communes_df,
                                     main_insee_commune,
                                     how='inner',
                                     left_on=['localisationco', 'dpt_code'],
                                     right_on=['NCCENR', 'DEP_id'])

# Create a new column 'insee' by combining NCCENR and insee_code
pas_communes_exact_dpt_df['insee'] = pas_communes_exact_dpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
pas_communes_exact_dpt_df = pas_communes_exact_dpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
pas_communes_exact_dpt_df = pas_communes_exact_dpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'dpt_exact'
pas_communes_exact_dpt_df['method'] = 'dpt_exact'

duplicates = pas_communes_exact_dpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    pas_communes_exact_dpt_df = pas_communes_exact_dpt_df[~duplicates]

# Update the linked_places_df dataframe
linked_places_df = pd.concat([linked_places_df, pas_communes_exact_dpt_df]).drop_duplicates()


In [24]:
# Tests
pas_communes_exact_dpt_df.head(5)
#pas_communes_exact_dpt_df.shape
pas_communes_exact_dpt_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,NCCENR_orig,insee_code,method
nunique,1587,1436,24,24,412,1008,1587,1437,1018,1009,1018,1
count,1587,1587,1587,1587,1587,1587,1587,1587,1587,1587,1587,1587
size,1587,1587,1587,1587,1587,1587,1587,1587,1587,1587,1587,1587


#### Without dpt

In [25]:
import pandas as pd

# Merge pas_communes_df and main_insee_commune on localisationco and NCCENR
pas_communes_exact_nodpt_df = pd.merge(pas_communes_df.loc[pas_communes_df['dpt_code'] == 'none'],
                                       main_insee_commune[main_insee_commune.DEP_id.isin(dpt_list)],
                                       how='inner',
                                       left_on='localisationco',
                                       right_on='NCCENR')

# Create a new column 'insee' by combining NCCENR and insee_code
pas_communes_exact_nodpt_df['insee'] = pas_communes_exact_nodpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
pas_communes_exact_nodpt_df = pas_communes_exact_nodpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
pas_communes_exact_nodpt_df = pas_communes_exact_nodpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'nodpt_exact'
pas_communes_exact_nodpt_df['method'] = 'nodpt_exact'

duplicates = pas_communes_exact_nodpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    pas_communes_exact_nodpt_df = pas_communes_exact_nodpt_df[~duplicates]

# Update the linked_places_df dataframe
linked_places_df = pd.concat([linked_places_df, pas_communes_exact_nodpt_df]).drop_duplicates()

In [26]:
# Test
pas_communes_exact_nodpt_df
#linked_places_df.shape
pas_communes_exact_nodpt_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,NCCENR_orig,insee_code,method
nunique,1,1,1,1,1,1,1,1,1,1,1,1
count,1,1,1,1,1,1,1,1,1,1,1,1
size,1,1,1,1,1,1,1,1,1,1,1,1


# Exact match avec noms simplifiés

**Traitement de données**

In [27]:
#on enlève les mots donnés par Olivier

In [28]:
def filter_words(text, tokens):
    text = replace_special_characters(text)  # Appliquer la fonction replace_special_characters
    words = text.split()
    words = [word for word in words if not re.search(r'\b{}\b'.format(re.escape(word.lower())), tokens)]
    filtered_text = ' '.join(words).strip() 
    return filtered_text

df['vedette'] = df['vedette'].apply(lambda x: filter_words(x, tokens=tokens))
df['localisationco'] = df['localisationco'].apply(lambda x: filter_words(x, tokens=tokens))

In [29]:
main_insee_commune['NCCENR'] = main_insee_commune['NCCENR'].apply(lambda x: filter_words(x, tokens=tokens))

# Liage des articles de type commune

In [30]:
# Extraction des articles de type commune (ceux qui n’ont pas de commune de localisation)

communes_df = df[df['localisationco'] == 'none']
#communes_df.shape

**with dpt**

In [31]:
import pandas as pd

# Merge communes_df and main_insee_commune on vedette and dpt_code
communes_exact_simple_dpt_df = pd.merge(communes_df,
                                 main_insee_commune,
                                 how='inner',
                                 left_on=['vedette', 'dpt_code'],
                                 right_on=['NCCENR', 'DEP_id'])

# Create a new column 'insee' by combining NCCENR and insee_code
communes_exact_simple_dpt_df['insee'] = communes_exact_simple_dpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
communes_exact_simple_dpt_df = communes_exact_simple_dpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
communes_exact_simple_dpt_df = communes_exact_simple_dpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'dpt_exact'
communes_exact_simple_dpt_df['method'] = 'dpt_exact_simple'

duplicates = communes_exact_simple_dpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    communes_exact_simple_dpt_df = communes_exact_simple_dpt_df[~duplicates]

# Filtrer les lignes de communes_exact_simple_dpt_df avec la méthode "dpt_exact_simple"
filtered_df = communes_exact_simple_dpt_df.loc[communes_exact_simple_dpt_df['method'] == 'dpt_exact_simple']

# Filtrer les article_id qui ne sont pas déjà présents dans linked_places_df
filtered_df = filtered_df[~filtered_df['article_id'].isin(linked_places_df['article_id'])]

# Ajouter les lignes filtrées à linked_places_df
linked_places_df = pd.concat([linked_places_df, filtered_df]).drop_duplicates()

**without dpt**

In [32]:
# Merge communes_df and main_insee_commune on vedette and NCCENR
communes_exact_simple_nodpt_df = pd.merge(communes_df.loc[communes_df['dpt_code'] == 'none'],
                                   main_insee_commune[main_insee_commune.DEP_id.isin(dpt_list)],
                                   how='inner',
                                   left_on='vedette',
                                   right_on='NCCENR')

# Create a new column 'insee' by combining NCCENR and insee_code
communes_exact_simple_nodpt_df['insee'] = communes_exact_simple_nodpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
communes_exact_simple_nodpt_df = communes_exact_simple_nodpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
communes_exact_simple_nodpt_df = communes_exact_simple_nodpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'nodpt_exact'
communes_exact_simple_nodpt_df['method'] = 'nodpt_exact_simple'

duplicates = communes_exact_simple_nodpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    communes_exact_simple_nodpt_df = communes_exact_simple_nodpt_df[~duplicates]

# Filtrer les lignes de communes_exact_simple_dpt_df avec la méthode "dpt_exact_simple"
filtered_df = communes_exact_simple_nodpt_df.loc[communes_exact_simple_nodpt_df['method'] == 'nodpt_exact_simple']

# Filtrer les article_id qui ne sont pas déjà présents dans linked_places_df
filtered_df = filtered_df[~filtered_df['article_id'].isin(linked_places_df['article_id'])]

# Ajouter les lignes filtrées à linked_places_df
linked_places_df = pd.concat([linked_places_df, filtered_df]).drop_duplicates()

# Liage des communes de localisation

In [33]:
# Extraction des articles appartenant à une commune (ceux ont une commune de localisation)

pas_communes_df = df[df['localisationco'] != 'none']
pas_communes_df

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig
0,PO9-00001,Abbas,Aveyron,12,Rodez,Druelle,Abbas Aveyron con de Rodez cn...,Abbas,Druelle
1,PO9-00002,Abbaye Nouvelle,Lot,46,Salviac,Leobard,Abbaye Nouvelle L Lot con de...,Abbaye[-Nouvelle],Léobard
2,PO9-00003,Abeille,Haute-Vienne,87,Châlus,Chalus,Abeille L Haute Vienne con de ...,Abeille,Châlus
4,PO9-00005,Abloux,Indre,36,Saint-Benoît-du-Sault,Gilles,Abloux Indre con de Saint Benoît ...,Abloux,Saint-Gilles
11,PO9-00012,Aglan,Lot,46,Puy-l’Évêque,Soturac,Aglan Lot con de Puy l Évêque ...,Aglan,Soturac
...,...,...,...,...,...,...,...,...,...
5315,PO9-05396,Yvernault,Indre,36,Argenton,Mosnay,Yvernault Indre con d Argenton ...,Yvernault,Mosnay
5316,PO9-05397,Begoux,Lot,46,Cahors,Cahors,Bégoux Lot con de Cahors cne ...,Bégoux,Cahors
5317,PO9-05398,Chauzas,Corrèze,19,Vigeois,Perpezac Noir,Chauzas Corrèze con de Vigeois ...,Chauzas,Perpezac-le-Noir
5318,PO9-05399,Neuglise,Allier,03,Neuilly-le-Réal,Bessay Allier,Neuglise Allier con de Neuilly le...,Neuglise,Bessay-sur-Allier


**with dpt**

In [34]:
# Merge pas_communes_df and main_insee_commune on localisationco and dpt_code
pas_communes_exact_simple_dpt_df = pd.merge(pas_communes_df,
                                     main_insee_commune,
                                     how='inner',
                                     left_on=['localisationco', 'dpt_code'],
                                     right_on=['NCCENR', 'DEP_id'])

# Create a new column 'insee' by combining NCCENR and insee_code
pas_communes_exact_simple_dpt_df['insee'] = pas_communes_exact_simple_dpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
pas_communes_exact_simple_dpt_df = pas_communes_exact_simple_dpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
pas_communes_exact_simple_dpt_df = pas_communes_exact_simple_dpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'dpt_exact'
pas_communes_exact_simple_dpt_df['method'] = 'dpt_exact_simple'

duplicates = pas_communes_exact_simple_dpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    pas_communes_exact_simple_dpt_df = pas_communes_exact_simple_dpt_df[~duplicates]

# Filtrer les lignes de communes_exact_simple_dpt_df avec la méthode "dpt_exact_simple"
filtered_df = pas_communes_exact_simple_dpt_df.loc[pas_communes_exact_simple_dpt_df['method'] == 'dpt_exact_simple']

# Filtrer les article_id qui ne sont pas déjà présents dans linked_places_df
filtered_df = filtered_df[~filtered_df['article_id'].isin(linked_places_df['article_id'])]

# Ajouter les lignes filtrées à linked_places_df
linked_places_df = pd.concat([linked_places_df, filtered_df]).drop_duplicates()

**without dpt**

In [35]:
import pandas as pd

# Merge pas_communes_df and main_insee_commune on localisationco and NCCENR
pas_communes_exact_simple_nodpt_df = pd.merge(pas_communes_df.loc[pas_communes_df['dpt_code'] == 'none'],
                                       main_insee_commune[main_insee_commune.DEP_id.isin(dpt_list)],
                                       how='inner',
                                       left_on='localisationco',
                                       right_on='NCCENR')

# Create a new column 'insee' by combining NCCENR and insee_code
pas_communes_exact_simple_nodpt_df['insee'] = pas_communes_exact_simple_nodpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
pas_communes_exact_simple_nodpt_df = pas_communes_exact_simple_nodpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
pas_communes_exact_simple_nodpt_df = pas_communes_exact_simple_nodpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'nodpt_exact'
pas_communes_exact_simple_nodpt_df['method'] = 'nodpt_exact_simple'

duplicates = pas_communes_exact_simple_nodpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    pas_communes_exact_simple_nodpt_df = pas_communes_exact_simple_nodpt_df[~duplicates]

# Filtrer les lignes de communes_exact_simple_dpt_df avec la méthode "dpt_exact_simple"
filtered_df = pas_communes_exact_simple_nodpt_df.loc[pas_communes_exact_simple_nodpt_df['method'] == 'nodpt_exact_simple']

# Filtrer les article_id qui ne sont pas déjà présents dans linked_places_df
filtered_df = filtered_df[~filtered_df['article_id'].isin(linked_places_df['article_id'])]

# Ajouter les lignes filtrées à linked_places_df
linked_places_df = pd.concat([linked_places_df, filtered_df]).drop_duplicates()


# Fuzzy method

#### With dpt

In [36]:
import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes avec dpt de localisation (dpt/fuzzy match)
communes_fuzzy_dpt_df = communes_df[~communes_df.article_id.isin(linked_places_df['article_id'])][communes_df.dpt_code != 'none']
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df['vedette']
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df.apply(
    lambda x: [
        f"{nccenr} ({nccenr_to_insee.get(nccenr, '')}) - {main_insee_commune[main_insee_commune['NCCENR'] == nccenr]['NCCENR_orig'].iloc[0]}"
        for nccenr in difflib.get_close_matches(
            x['vedette'],
            main_insee_commune[main_insee_commune['DEP_id'] == x['dpt_code']]['NCCENR'],
            n=3
        )
        if all(word in nccenr.split() for word in x['vedette'].split())
    ],
    axis=1
)

# Sortir les communes non liées
communes_fuzzy_dpt_df = communes_fuzzy_dpt_df[communes_fuzzy_dpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df['insee_code'].apply(lambda x: ', '.join(x))

communes_fuzzy_dpt_df['method'] = 'dpt_fuzzy'

# Actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, communes_fuzzy_dpt_df]).drop_duplicates()


/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_2058/1538882507.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  communes_fuzzy_dpt_df = communes_df[~communes_df.article_id.isin(linked_places_df['article_id'])][communes_df.dpt_code != 'none']


In [37]:
# Tests
#communes_fuzzy_dpt_df.head(20)
#linked_places_df.shape
#linked_places_df
communes_fuzzy_dpt_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,insee_code,method
nunique,57,57,18,18,48,1,57,57,1,56,1
count,57,57,57,57,57,57,57,57,57,57,57
size,57,57,57,57,57,57,57,57,57,57,57


In [38]:
communes_fuzzy_dpt_df

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,insee_code,method
89,PO9-00091,Anglards,Cantal,15,Salers,none,Anglards Cantal con de Salers ...,Anglards,none,"Freix Anglards (15072) - Freix-Anglards, Angla...",dpt_fuzzy
127,PO9-00129,Argenton,Indre,36,Châteauroux,none,Argenton Indre con de Châteauroux...,Argenton,none,Argenton Creuse (36006) - Argenton-sur-Creuse,dpt_fuzzy
220,PO9-00224,Auteyrac,Haute-Loire,43,Langeac,none,Auteyrac Haute Loire con de Lange...,Auteyrac,none,Vissac Auteyrac (43013) - Vissac-Auteyrac,dpt_fuzzy
339,PO9-00344,Beaulieu,Loiret,45,Châtillon-sur-Loire,none,Beaulieu Loiret con de Châtillon ...,Beaulieu,none,Beaulieu Loire (45029) - Beaulieu-sur-Loire,dpt_fuzzy
349,PO9-00355,Beaumont,Haute-Vienne,87,Eymoutiers,none,Beaumont Haute Vienne con d Eymou...,Beaumont,none,Beaumont Lac (87009) - Beaumont-du-Lac,dpt_fuzzy
382,PO9-00389,Belleville,Cher,18,Léré,none,Belleville Cher con de Léré ...,Belleville,none,Belleville Loire (18026) - Belleville-sur-Loire,dpt_fuzzy
563,PO9-00573,Bournoncle,Haute-Loire,43,Brioude,none,Bournoncle Haute Loire con de Bri...,Bournoncle,none,Bournoncle Pierre (43038) - Bournoncle-Saint-P...,dpt_fuzzy
891,PO9-00905,Cernoy,Loiret,45,Châtillon-sur-Loire,none,Cernoy Loiret con de Châtillon su...,Cernoy,none,Cernoy Berry (45064) - Cernoy-en-Berry,dpt_fuzzy
949,PO9-00964,Champagnac,Haute-Loire,43,Auzon,none,Champagnac Haute Loire con d Auzo...,Champagnac,none,Champagnac Vieux (43052) - Champagnac-le-Vieux,dpt_fuzzy
970,PO9-00985,Champs Tarentaine,Cantal,15,Mauriac,none,Champs sur Tarentaine Cantal con ...,Champs-sur-Tarentaine,none,Champs Tarentaine Marchal (15038) - Champs-sur...,dpt_fuzzy


#### Without dpt

In [39]:
import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes sans dpt de localisation (fuzzy match)
communes_fuzzy_nodpt_df = communes_df[~communes_df.article_id.isin(linked_places_df['article_id'])][communes_df.dpt_code == 'none']
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df['vedette']
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df.apply(
    lambda x: [
        f"{nccenr} ({nccenr_to_insee.get(nccenr, '')}) - {main_insee_commune[main_insee_commune['NCCENR'] == nccenr]['NCCENR_orig'].iloc[0]}"
        for nccenr in difflib.get_close_matches(
            x['vedette'],
            main_insee_commune['NCCENR'],
            n=3
        )
        if all(word in nccenr.split() for word in x['vedette'].split())
    ],
    axis=1
)

# Sortir les communes non liées
communes_fuzzy_nodpt_df = communes_fuzzy_nodpt_df[communes_fuzzy_nodpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df['insee_code'].apply(lambda x: ', '.join(x))

communes_fuzzy_nodpt_df['method'] = 'nodpt_fuzzy'

# Actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, communes_fuzzy_nodpt_df]).drop_duplicates()


/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_2058/279753277.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  communes_fuzzy_nodpt_df = communes_df[~communes_df.article_id.isin(linked_places_df['article_id'])][communes_df.dpt_code == 'none']


In [40]:
# Tests
communes_fuzzy_nodpt_df
#communes_fuzzy_nodpt_df.shape
#linked_places_df.shape
communes_fuzzy_nodpt_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,insee_code,method
nunique,6,6,3,1,2,1,6,6,1,6,1
count,6,6,6,6,6,6,6,6,6,6,6
size,6,6,6,6,6,6,6,6,6,6,6


In [41]:
communes_fuzzy_nodpt_df

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,insee_code,method
58,PO9-00060,Allier,none,none,none,none,Allier rivière affluent de la Loire ...,Allier,none,Allier (65005) - Allier,nodpt_fuzzy
2176,PO9-02202,Lescar,Basses-Pyrénées,none,Pau,none,Lescar Basses Pyrénées con de Pau...,Lescar,none,Lescar (64335) - Lescar,nodpt_fuzzy
2240,PO9-02267,Livradois,none,none,none,none,Livradois région Libratensis ...,Livradois,none,Marsac Livradois (63211) - Marsac-en-Livradois,nodpt_fuzzy
3329,PO9-03371,Riviere,none,none,none,none,Rivière La voyage du diocèse de Rod...,Rivière,none,"Riviere (97211) - Rivière, Riviere (97211) - R...",nodpt_fuzzy
3672,PO9-03719,Denis,Seine,none,none,none,Saint Denis Seine Sanctus Diony...,Saint-Denis,none,"Denis (97411) - Saint-Denis, Denis (97411) - S...",nodpt_fuzzy
5082,PO9-05156,Vaux,none,none,none,none,Vaux Les archidiaconé et archiprêtr...,Vaux,none,"Vaux (86278) - Vaux, Vaux (86278) - Vaux, Vaux...",nodpt_fuzzy


# Fuzzy match method

#### With dpt

In [42]:
import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes avec dpt de localisation (dpt/fuzzy match)
pas_communes_fuzzy_dpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_places_df['article_id'])][pas_communes_df.dpt_code != 'none']
pas_communes_fuzzy_dpt_df['insee_code'] = pas_communes_fuzzy_dpt_df['localisationco']
pas_communes_fuzzy_dpt_df['insee_code'] = pas_communes_fuzzy_dpt_df.apply(
    lambda x: [
        f"{nccenr} ({nccenr_to_insee.get(nccenr, '')}) - {main_insee_commune[main_insee_commune['NCCENR'] == nccenr]['NCCENR_orig'].iloc[0]}"
        for nccenr in difflib.get_close_matches(
            x['localisationco'],
            main_insee_commune[main_insee_commune['DEP_id'] == x['dpt_code']]['NCCENR'],
            n=3
        )
        if all(word in nccenr.split() for word in x['localisationco'].split())
    ],
    axis=1
)

# Sortir les communes non liées
pas_communes_fuzzy_dpt_df = pas_communes_fuzzy_dpt_df[pas_communes_fuzzy_dpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
pas_communes_fuzzy_dpt_df['insee_code'] = pas_communes_fuzzy_dpt_df['insee_code'].apply(lambda x: ', '.join(x))

pas_communes_fuzzy_dpt_df['method'] = 'dpt_fuzzy'

# Actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, pas_communes_fuzzy_dpt_df]).drop_duplicates()


/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_2058/1249108763.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  pas_communes_fuzzy_dpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_places_df['article_id'])][pas_communes_df.dpt_code != 'none']


In [43]:
# Tests
pas_communes_fuzzy_dpt_df
#linked_places_df.shape
#linked_places_df
pas_communes_fuzzy_dpt_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,insee_code,method
nunique,38,38,11,11,28,29,38,38,29,29,1
count,38,38,38,38,38,38,38,38,38,38,38
size,38,38,38,38,38,38,38,38,38,38,38


In [44]:
pas_communes_fuzzy_dpt_df

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,insee_code,method
88,PO9-00090,Anglard,Puy-de-Dôme,63,Besse,Anastaise,Anglard Puy de Dôme con de Besse ...,Anglard,Saint-Anastaise,Besse et Anastaise (63038) - Besse-et-Saint-An...,dpt_fuzzy
301,PO9-00305,Bars,Aveyron,12,Mur-de-Barrez,Lacroix,Bars Aveyron con de Mur de Barrez...,Bars,Lacroix,Lacroix Barrez (12118) - Lacroix-Barrez,dpt_fuzzy
311,PO9-00315,Bastide,Cantal,15,Aurillac-sud,Arpajon,Bastide La Cantal con d Aurilla...,Bastide,Arpajon,Arpajon Cere (15012) - Arpajon-sur-Cère,dpt_fuzzy
424,PO9-00431,Beteille,Aveyron,12,Najac,Andre,Béteille Aveyron con de Najac ...,Béteille,Saint-André,Andre Najac (12210) - Saint-André-de-Najac,dpt_fuzzy
647,PO9-00658,Bruejouls,Aveyron,12,Marcillac-Vallon,Clairvaux,Bruéjouls Aveyron con de Marcilla...,Bruéjouls,Clairvaux,Clairvaux Aveyron (12066) - Clairvaux-d'Aveyron,dpt_fuzzy
723,PO9-00737,Cambon,Aveyron,12,Saint-Affrique,Vabres,Cambon Le Aveyron con de Saint ...,Cambon,Vabres,Vabres Abbaye (12286) - Vabres-l'Abbaye,dpt_fuzzy
1274,PO9-01293,Conros,Cantal,15,Aurillac-sud,Arpajon,Conros Cantal con d Aurillac sud ...,Conros,Arpajon,Arpajon Cere (15012) - Arpajon-sur-Cère,dpt_fuzzy
1336,PO9-01356,Courtille,Allier,03,Souvigny,Noyant,Courtille Allier con de Souvigny ...,Courtille,Noyant,Noyant Allier (03202) - Noyant-d'Allier,dpt_fuzzy
1428,PO9-01449,Dampierre,Indre,36,Éguzon,Gargilesse,Dampierre Indre con d Éguzon ...,Dampierre,Gargilesse,Gargilesse Dampierre (36081) - Gargilesse-Damp...,dpt_fuzzy
1815,PO9-01839,Gleyse Nove,Aveyron,12,Vézins,Vezins,Gleyse Nove Aveyron con de Vézins...,Gleyse-Nove,Vézins,Vezins Levezou (12294) - Vézins-de-Lévézou,dpt_fuzzy


#### Without dpt

In [45]:
import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes sans dpt de localisation (fuzzy match)
pas_communes_fuzzy_nodpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_places_df['article_id'])][pas_communes_df.dpt_code == 'none']
pas_communes_fuzzy_nodpt_df['insee_code'] = pas_communes_fuzzy_nodpt_df['localisationco']
pas_communes_fuzzy_nodpt_df['insee_code'] = pas_communes_fuzzy_nodpt_df.apply(
    lambda x: [
        f"{nccenr} ({nccenr_to_insee.get(nccenr, '')}) - {main_insee_commune[main_insee_commune['NCCENR'] == nccenr]['NCCENR_orig'].iloc[0]}"
        for nccenr in difflib.get_close_matches(
            x['localisationco'],
            main_insee_commune['NCCENR'],
            n=3
        )
        if all(word in nccenr.split() for word in x['localisationco'].split())
    ],
    axis=1
)

# Sortir les communes non liées
pas_communes_fuzzy_nodpt_df = pas_communes_fuzzy_nodpt_df[pas_communes_fuzzy_nodpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
pas_communes_fuzzy_nodpt_df['insee_code'] = pas_communes_fuzzy_nodpt_df['insee_code'].apply(lambda x: ', '.join(x))

pas_communes_fuzzy_nodpt_df['method'] = 'nodpt_fuzzy'

# Actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, pas_communes_fuzzy_nodpt_df]).drop_duplicates()


/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_2058/3054842666.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  pas_communes_fuzzy_nodpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_places_df['article_id'])][pas_communes_df.dpt_code == 'none']


In [46]:
pas_communes_fuzzy_nodpt_df

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,insee_code,method
3892,PO9-03943,Germain Pres,none,none,none,Paris,Saint Germain des Prés cne de Paris...,Saint-Germain-des-Prés,Paris,Paris (75056) - Paris,nodpt_fuzzy
3897,PO9-03948,Germain Auxerrois,none,none,none,Paris,Saint Germain l Auxerrois cne de Pari...,Saint-Germain-l’Auxerrois,Paris,Paris (75056) - Paris,nodpt_fuzzy


In [47]:
# Tests
pas_communes_fuzzy_nodpt_df
#linked_places_df.shape
pas_communes_fuzzy_nodpt_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,insee_code,method
nunique,2,2,1,1,1,1,2,2,1,1,1
count,2,2,2,2,2,2,2,2,2,2,2
size,2,2,2,2,2,2,2,2,2,2,2


## Rapport et analyse

In [48]:
#on append les deux dataframes ensemble

new_rows_df = df[~df['article_id'].isin(linked_places_df['article_id'])]

if 'method' not in new_rows_df.columns:
    new_rows_df['method'] = 'nulle'

linked_places_df = linked_places_df.append(new_rows_df, ignore_index=True)


/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_2058/170212847.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_rows_df['method'] = 'nulle'
/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_2058/170212847.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linked_places_df = linked_places_df.append(new_rows_df, ignore_index=True)


In [49]:
linked_places_df

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,NCCENR_orig,insee_code,method
0,PO9-00006,Abrest,Allier,03,Vichy,none,Abrest Allier con de Vichy ...,Abrest,none,Abrest,Abrest (03001),dpt_exact
1,PO9-00007,Abzac,Charente,16,Confolens-sud,none,Abzac Charente con de Confolens s...,Abzac,none,Abzac,Abzac (16001),dpt_exact
2,PO9-00008,Acheres,Cher,18,Henrichemont,none,Achères Cher con d Henrichemont ...,Achères,none,Achères,Acheres (18001),dpt_exact
3,PO9-00009,Affieux,Corrèze,19,Treignac,none,Affieux Corrèze con de Treignac ...,Affieux,none,Affieux,Affieux (19001),dpt_exact
4,PO9-00010,Agen,Lot-et-Garonne,47,none,none,Agen Lot et Garonne Agennense ...,Agen,none,Agen,Agen (47001),dpt_exact
...,...,...,...,...,...,...,...,...,...,...,...,...
5316,PO9-05346,Vindrac,Tarn,81,Cordes,Vindrac Aleyrac,Vindrac Tarn con de Cordes cn...,Vindrac,Vindrac-Aleyrac,NaN,NaN,nulle
5317,PO9-05358,Visac,Tarn,81,Albi,Lescure,Visac Le Tarn con d Albi cn...,Visac,Lescure,NaN,NaN,nulle
5318,PO9-05359,Vissac,Haute-Loire,43,Langeac,none,Vissac Haute Loire con de Langeac...,Vissac,none,NaN,NaN,nulle
5319,PO9-05378,Vors,Aveyron,12,Rodez,none,Vors Aveyron con de Rodez V...,Vors,none,NaN,NaN,nulle


In [50]:
linked_places_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,NCCENR_orig,insee_code,method
nunique,5320,4823,39,37,577,1184,5321,4806,1191,3298,3511,6
count,5321,5321,5321,5321,5321,5321,5321,5321,5321,4885,4988,5321
size,5321,5321,5321,5321,5321,5321,5321,5321,5321,5321,5321,5321


In [51]:
# des doublons?
nombre_de_doublons = linked_places_df.duplicated(subset=['article_id']).sum()
print(nombre_de_doublons)


1


In [52]:
linked_places_df.loc[linked_places_df.duplicated(keep=False,subset=['article_id']), :]

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco,reference,vedette_orig,localisationco_orig,NCCENR_orig,insee_code,method
2609,PO9-04389,Saint Paulien,Haute-Loire,43,Puy,none,Saint Paulien Haute Loire con du ...,Saint-Paulien,none,Saint-Paulien,Saint Paulien (43216),dpt_exact
4658,PO9-04389,Notre Dame du Haut Solier,Haute-Loire,43,Puy,Saint Paulien,Notre Dame du Haut Solier Haute Loire...,Notre-Dame du Haut-Solier,Saint-Paulien,Saint-Paulien,Saint Paulien (43216),dpt_exact


In [53]:
method_counts = linked_places_df['method'].value_counts()
print(method_counts)

dpt_exact           4813
nulle                333
dpt_fuzzy             95
dpt_exact_simple      68
nodpt_fuzzy            8
nodpt_exact            4
Name: method, dtype: int64


In [54]:
vedette_list = pd.unique(linked_places_df['vedette'])
linked_places_df['sort_key'] = linked_places_df.apply(lambda x: x['localisationco'] if x['localisationco'] in vedette_list else x['vedette'], axis=1)
linked_places_df.sort_values(['method', 'sort_key', 'article_id'], inplace=True)
linked_places_df = linked_places_df.drop('sort_key', axis=1)
linked_places_df = linked_places_df[['article_id', 'vedette', 'vedette_orig', 'localisationde', 'dpt_code', 'canton_code', 'localisationco', 'localisationco_orig', 'NCCENR_orig', 'insee_code', 'method', 'reference']]
linked_places_df

,article_id,vedette,vedette_orig,localisationde,dpt_code,canton_code,localisationco,localisationco_orig,NCCENR_orig,insee_code,method,reference
3229,PO9-00001,Abbas,Abbas,Aveyron,12,Rodez,Druelle,Druelle,Druelle,Druelle (12090),dpt_exact,Abbas Aveyron con de Rodez cn...
3232,PO9-00002,Abbaye Nouvelle,Abbaye[-Nouvelle],Lot,46,Salviac,Leobard,Léobard,Léobard,Leobard (46169),dpt_exact,Abbaye Nouvelle L Lot con de...
0,PO9-00006,Abrest,Abrest,Allier,03,Vichy,none,none,Abrest,Abrest (03001),dpt_exact,Abrest Allier con de Vichy ...
1,PO9-00007,Abzac,Abzac,Charente,16,Confolens-sud,none,none,Abzac,Abzac (16001),dpt_exact,Abzac Charente con de Confolens s...
2,PO9-00008,Acheres,Achères,Cher,18,Henrichemont,none,none,Achères,Acheres (18001),dpt_exact,Achères Cher con d Henrichemont ...
...,...,...,...,...,...,...,...,...,...,...,...,...
5318,PO9-05359,Vissac,Vissac,Haute-Loire,43,Langeac,none,none,NaN,NaN,nulle,Vissac Haute Loire con de Langeac...
5110,PO9-02026,Lac,Lac,Aveyron,12,Rodez,Vors,Vors,NaN,NaN,nulle,Lac Le Aveyron con de Rodez ...
5119,PO9-02180,Lax,Lax,Aveyron,12,Rodez,Vors,Vors,NaN,NaN,nulle,Lax Aveyron con de Rodez cne ...
5319,PO9-05378,Vors,Vors,Aveyron,12,Rodez,none,none,NaN,NaN,nulle,Vors Aveyron con de Rodez V...


In [55]:
empty_rows = linked_places_df[linked_places_df['vedette'] == '']
empty_rows

,article_id,vedette,vedette_orig,localisationde,dpt_code,canton_code,localisationco,localisationco_orig,NCCENR_orig,insee_code,method,reference


In [56]:
empty_rows.agg(['nunique', 'count', 'size'])

,article_id,vedette,vedette_orig,localisationde,dpt_code,canton_code,localisationco,localisationco_orig,NCCENR_orig,insee_code,method,reference
nunique,0,0,0,0,0,0,0,0,0,0,0,0
count,0,0,0,0,0,0,0,0,0,0,0,0
size,0,0,0,0,0,0,0,0,0,0,0,0


## Export

In [57]:
# Sauvegarder le dataframe fusionné
linked_places_df.to_csv('../../utils/pouille/out/linking_out/liage_po9.csv', index=False)
